In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import copy
import os

import numpy as np
import torch
import torch.distributions as dist
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
import seaborn as sns

import flowtorch as ft

import nflows
import normflow as nf

from signatureshape.animation import fetch_animation_id_set, fetch_animations
from signatureshape.animation.src.mayavi_animate import mayavi_animate

from deepthermal.validation import (
    create_subdictionary_iterator,
    k_fold_cv_grid,
    add_dictionary_iterators,
)

from deepthermal.FFNN_model import fit_FFNN
from deepthermal.plotting import plot_result

import shapeflow as sf

# make reproducible
seed = torch.manual_seed(0)

# better formats
set_matplotlib_formats("pdf", "svg")

In [ ]:
# fetch data as so3
# we assume all have the same skeleton
print("Loading mocap data:")
# walk  data
walk_subjects = ["07", "08", "35", "16"]
walk_animations = []
walk_desc = []
for s in walk_subjects:
    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:4] == "walk":
            walk_animations.append(t[1])
            walk_desc.append(t[2])

walk_animations_train_frame = sum(
    len(anim.get_frames()) for anim in walk_animations[:18]
)

# run data
run_subjects = ["09", "16", "35"]
run_animations = []
run_skeletons = []
for s in run_subjects:

    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:3] == "run":
            run_skeletons.append(t[0])
            run_animations.append(t[1])

print("Convert to array:")
walk_angle_array = sf.utils.animation_to_eulers(
    walk_animations, reduce_shape=True, remove_root=True, deg2rad=True
)
run_angle_array = sf.utils.animation_to_eulers(
    run_animations, reduce_shape=True, remove_root=True, deg2rad=True
)

In [ ]:
# save data since it takes so long to get

np.save("walk_angle_array.npy", walk_angle_array)
walk_angle_tensor = torch.tensor(
    np.load("walk_angle_array.npy", allow_pickle=False)
).float()

np.save("run_angle_array.npy", run_angle_array)
run_angle_tensor = torch.tensor(
    np.load("run_angle_array.npy", allow_pickle=False)
).float()

In [ ]:
data = torch.utils.data.TensorDataset(walk_angle_tensor)
data_run = torch.utils.data.TensorDataset(run_angle_tensor)

In [ ]:
#######
DIR = "../figures/frames/"
SET_NAME = "walk_residual"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########
FOLDS = 5

event_shape = data[0][0].shape
base_dist = dist.Independent(
    dist.Normal(loc=torch.zeros(event_shape), scale=torch.ones(event_shape)), 1
)

lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode="min", factor=0.5, patience=10, verbose=True
)
# def get_flow(ji)
MODEL_PARAMS = {
    "model": [sf.utils.get_flow],
    "get_transform": [sf.normalizing_flows.get_residual_transform],
    "base_dist": [base_dist],
    # "input_dim" : [event_shape[0]],
    "inverse_model": [False],
    "compose": [True],
}
num_layers = 5
MODEL_PARAMS_EXPERIMENT = {
    "hidden_features": [[47] * num_layers, [47 * 2] * num_layers],
    "hidden_layers": [[2] * num_layers, [2] * num_layers],
    "n_exact_terms": [[4] * num_layers],
    "n_samples": [[10] * num_layers],
}

TRAINING_PARAMS = {
    "batch_size": [5000],
    "regularization_param": [1e-5],
    "compute_loss": [sf.monte_carlo_dkl_loss],
    "post_batch": [sf.get_post_step_lipchitz(5)],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "verbose_interval": [20],
    "optimizer": ["ADAM"],
    "num_epochs": [1000],
    "learning_rate": [0.1],
    "lr_scheduler": [lr_scheduler],
}

In [ ]:
# create iterators
model_params_iter_1 = create_subdictionary_iterator(MODEL_PARAMS)

model_exp_iter = create_subdictionary_iterator(MODEL_PARAMS_EXPERIMENT, product=False)
exp_model_params_iter = add_dictionary_iterators(model_exp_iter, model_params_iter_1)

training_params_iter = create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = create_subdictionary_iterator(
    TRAINING_PARAMS_EXPERIMENT, product=False
)
exp_training_params_iter = add_dictionary_iterators(
    training_exp_iter, training_params_iter
)

In [ ]:
cv_results = k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=fit_FFNN,
    training_params=exp_training_params_iter,
    data=data,
    folds=FOLDS,
    verbose=True,
    trials=1,
    partial=True,
    shuffle_folds=False,
)

Test that the wrapper works

In [ ]:
plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
    # plot_function=plot_model_1d,
    # function_kwargs=plot_kwargs,
)

In [ ]:
flow = cv_results["models"][0][0]

In [ ]:
torch.save(flow, "Flow_frames" + SET_NAME + "2.pt")

In [ ]:
noise = base_dist.sample([100])
print("Log vals:")

print("Noise :", flow.log_prob(noise[0:1]).mean().item())
print("Trian data:", flow.log_prob(data[:][0]).mean().item())

In [ ]:
sample = flow.sample([1])

In [ ]:
len(skel.bones)

In [ ]:
# get_two motions
# x_first_frame = torch.tensor(np.deg2rad(walk_animations[0].to_numpy_array())[3:].T).float()[0:1]
# x_first_frame = torch.tensor(np.deg2rad(walk_animations[0].to_numpy_array())[3:].T).float()[0:1]
i, j = 0, 1200
x_first_frame = data[i : i + 1][0]
x_second_frame = data[j : j + 1][0]

In [ ]:
# interpolate
z_first_walk = flow.bijector.inverse(x_first_frame)
z_second_walk = flow.bijector.inverse(x_second_frame)

In [ ]:
w_list = torch.cat((torch.linspace(1, 0, 120), torch.linspace(0, 1, 120)))
x_interpolated = torch.cat(
    [flow.bijector.forward(z_first_walk * w + z_second_walk * (1 - w)) for w in w_list]
)
x_interpolated_test = torch.cat(
    [x_first_frame * w + x_second_frame * (1 - w) for w in w_list]
)

In [ ]:
skel = copy.deepcopy(run_skeletons[0])

anim_test = copy.deepcopy(walk_animations[0])
anim_first = copy.deepcopy(walk_animations[0])
anim_second = copy.deepcopy(walk_animations[0])
anim_first.from_numpy_array(sf.utils.data_to_motion_array(x_first_frame))
anim_second.from_numpy_array(sf.utils.data_to_motion_array(x_second_frame))
anim_test.from_numpy_array(sf.utils.data_to_motion_array(x_interpolated))

In [ ]:
anim = mayavi_animate(
    skel,
    anim_first,
    offset=[0, 0, 0],
    continuous=False,
    fixed_cam=False,
    frame_limit=-1,
    save_path=None,
)

In [ ]:
anim = mayavi_animate(
    skel,
    anim_second,
    offset=[0, 0, 0],
    continuous=False,
    fixed_cam=False,
    frame_limit=-1,
    save_path=None,
)

In [ ]:
anim = mayavi_animate(
    skel,
    anim_test,
    offset=[0, 0, 0],
    continuous=False,
    fixed_cam=False,
    frame_limit=-1,
    save_path=None,
)